In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import intel_npu_acceleration_library
from intel_npu_acceleration_library.compiler import CompilerConfig
from intel_npu_acceleration_library.dtypes import int8
import torch
from torch.utils.data import Dataset
import time
import json
import os
from tqdm import tqdm

c:\Users\Girish\.conda\envs\venv\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Girish\.conda\envs\venv\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# # Image Prompt Dataset
class CaptionDataset(Dataset):
    def __init__(self, captions_file):
        with open(captions_file, 'r') as f:
            self.data = json.load(f)
        
        self.image_filenames = list(self.data.keys())
        self.captions = list(self.data.values())
    
    def __len__(self):
        return len(self.image_filenames)
    
    def __getitem__(self, idx):
        image_filename = self.image_filenames[idx]
        caption = self.captions[idx]
        return image_filename, caption

In [3]:
# Dataset
captions_file = r"D:\Work\College\img_prompts_part_2.json"
result_file = r"D:\Work\College\merged_captions_part_2.json"
if not os.path.exists(result_file):    
    with open(result_file, "w", encoding="utf-8") as f:
        ...
dataset = CaptionDataset(captions_file)
print(len(dataset))

10000


In [4]:
# Load model and tokenizer
# tokenizer_name = "nvidia/Nemotron-Mini-4B-Instruct"
tokenizer_name = r"D:\Work\College\nemotron_tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_default_system_prompt=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

# model_name = "nvidia/Nemotron-Mini-4B-Instruct"
# model_name = "nemotron_model"
# model = AutoModelForCausalLM.from_pretrained(model_name).eval()
# model = intel_npu_acceleration_library.compile(model, CompilerConfig(dtype=int8))


# Save model and tokenizer
# tokenizer.save_pretrained("nemotron_tokenizer")
# model.save_pretrained("nemotron_model")
# torch.save(model, "nemotron_model.pth")
model = torch.load("nemotron_model_npu.pth")

C:\Users\Girish\AppData\Local\Temp\ipykernel_19824\2754742975.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("nemotron_model_npu.pth")


In [ ]:
def generate_caption(id):
    pth, prompt = dataset[id]

    # prompt = (
    #     "The following captions describe the same image in different ways. "
    #     "Merge them into a single, clear, and accurate description of the same scene, avoiding repetition:\n\n"
    #     + "\n".join(lst)
    #     + "\n\nMerged Paragraph:"
    # )

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate output
    with torch.inference_mode():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and clean up output
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    generated_paragraph = output_text.split("Merged Paragraph:")[-1].strip()
    
    return pth, generated_paragraph

In [6]:
with open(result_file, "r", encoding="utf-8") as f:
    checkpoint = len(f.readlines()) - 1

print(checkpoint)

4160


In [7]:
with open(result_file, "a", encoding="utf-8") as f:
    if checkpoint == -1:
      f.write("{\n")  # Start JSON object
      first_entry = True
    else:
      first_entry = False

    for i in tqdm(range(5000), total=5000, desc="Generating Captions"):
        if i < checkpoint:
            continue

        img, generated_caption = generate_caption(i)

        # Write each entry separately
        if not first_entry:
          f.write(",\n")  # Add a comma before new entries (except the first)
        json.dump(img, f)
        f.write(": ")
        json.dump(generated_caption, f)

        f.flush()  # Ensure data is written immediately
        first_entry = False

    f.write("\n}")  # End JSON object

Generating Captions: 100%|██████████| 5000/5000 [3:08:39<00:00,  2.26s/it]  
